# Proyecto Final Área Imágenes: 
EL5206 Laboratorio de Inteligencia Computacional y Robótica
- Profesor: Claudio Pérez F
- Profesor Auxiliar: Juan Pablo Pérez
- Estudiantes: Pablo Yáñez M, Magdalena De La Fuente

## Content Based Image Retrieval (CBIR)
El Content Based Image Retrieval consiste en una técnica para la búsqueda de imágenes dentro de una base de datos a partir de las características presentes en las imágenes. Estas características pueden ser extraídas a través de distintos algoritmos. 

* **Objetivo del Proyecto**

El objetivo de este proyecto final es desarrollar e implementar un algoritmo de búsqueda de imágenes similares basado en su contenido (Content Based Image Retrieval, CBIR) utilizando distintos métodos de extracción de características. 

* **Sobre las imágenes**

Se utilizan los datasets INRIA Holidays dataset y GPR1200. El primero consiste en imágenes de consulta para 500 clases, mientras que el segundo consta de 1200 clases con 10 ejemplos por clase. 

* **Sobre el trabajo realizado**
  
En este informe se aborda la implementación de distintos métodos para atacat la problemática CBIR. 

El primer método consiste en la construcción de un método de obtención de vectores característicos "handcrafted", en donde se utiliza un modelo de Bag of Visual Words. Este obtiene descriptores provisos por el algoritmo SIFT, luego clusteriza estos descriptores y finalmente encuentra las palabras visuales. Esta información es finalmente proyectada en un histograma, y estos histogramas se almacenan en un archivos .csv. Por cada imagen del dataset hay un histograma. 

El segundo método implementado utiliza redes neuronales convolucionales para generar vectores característicos. Se proveen las imágenes a la red XXX rellenar con detalles. 

El tercer método implementado correponde a la fusión de los vectores característicos del método handcrafter y el método por CNN. Estos vectores se concatenan para formar un solo vector característico por imagen, y luego son enviados a un proceso de LDA para reducir la dimensionalidad, obteniéndose finalmente, un vector unimidensional más corto que CNN pero más largo que a través del método hand crafted.

El cuarto?

* **Sobre el código implementado**

Todas las funciones implementadas han sido empaquetadas para hacer uso de estas de manera fácil en múltiples archivos o notebooks. Los modulos se pueden encontrar en la carpeta src del respositorio. 



In [ ]:
 
import pandas as pd
from PIL import Image 
from src.SIFT.SIFT_gen_and_utils import SIFTFeatures 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
from src.Metrics.metrics_utils import get_hist_from_str, plot_10,plot_histogram,similarity_metric, query_image, evaluate_query
from src.SIFT.SIFT_gen_and_utils import SIFTFeatures 

In [41]:
df = pd.read_csv('/root/labint/LabInt/LabInt/final_histograms.csv')
SIFT = SIFTFeatures("", "", run=False)

KeyboardInterrupt: 

In [ ]:
df["class"] = df["image_name"].apply(lambda x: SIFT.get_class(x))
df["features_histogram"] = df["features_histogram"].apply(lambda x: get_hist_from_str(x, separator=' '))
df["features_fusion"] = df["features_fusion"].apply(lambda x: get_hist_from_str(x, separator=','))

In [ ]:
#df['features_CNN'][0]

In [ ]:
#PARAMETROS GLOBALES
IMAGE_EXAMPLE = df["image_name"][0]
MEASURE = 'cosine'
PATH_TO_IMAGES = '/root/labint/LabInt/LabInt/data/GPR1200/images'

### Método Handcrafted

El método handcrafted consiste de cuatro pasos:

#### 1. Generación de decriptores por SIFT

El algoritmo *Scale Invariant Feature Transform (SIFT)* es un algoritmo de visión computacional de extraccion de características para imágenes. Como dice su nombre, es invariante a la escala y rotación de una imagen. Se utiliza en una variedad de aplicaciones, entre ellas el reconocimiento de objetos, mapeo robótico, navegación, modelamiento 3d, etc. 

Este algoritmo retorna una lista de valores descriptores que corresponden a posiciones en la imagen que cumplen alguna particularidad.  En esta ocasión se fijó el numero de descriptores a generar por imagen en 10000. Se utiliza la funcion `cv2.SIFT_create` y luego `sift.detectAndCompute` para calcularle los descriptores a cada imagen. Como mencionado antes la ventaja de utilizar SIFT es la invarianza de los descriptores a la rotación y escalamiento, sin embargo, una desventaja es que no es sensible al color, ya que las imágenes son convertidas a escala de grises antes de computar los descriptores. Esto a la vez lo vuelve sensible al constraste de la imagen. 

#### 2. Obtención de clusters por Kmeans 
Debido al alto volumen de datos generado por SIFT, solo se utiliza un subconjunto de imagenes para construir los clusteres de palabras. Por supuesto, el subconjunto a sido balanceado y estratificado para evitar problemas asociados al desbalance de clases. Se utilizó un subsampling de 0.35, es decir, 35% de la data total. Numéricamente esto corresponde a 472185 imágenes de un total de 13491. 

En Kmeans se debe elegir un número de clusters a encontrar por el algoritmo. Uno de los métodos más comunes para seleccionar el número de clusters es el método del codo. Este consiste en encontrar el punto de inflexión en una curva de pérdida SSE (sum of squared errors).

En esta ocasión el número de clusters es 120. 

#### 3. Cuantización Obtencion de palabras visuales
Para obtener las palabras visuales se obtienen los 120 centroides generados en la sección 2. Luego, para la data no perteneciente al subconjunto mencionado antes, se asocia cada datapoint a el cluster correspondiente. Esta metodología permite reducir significativamente los tiempos de computo para un dataset de gran tamaño.   

#### 4. Construcción de histogramas
El último paso en este método es la construcción de histogramas. Se utiliza la funcion `np.histogram` de numpy para construir un histograma donde el número de bins es el número de clusters obtenido por el método del codo anteriormente.



#### Plot de un histograma handcrafted

A continuación se muestra el histograma generado para una imagen del dataset utilizando el método handcrafted. Tiene 120 bins, correspondiente al número de clusters señalado por el método del codo.

In [ ]:
plot_histogram(df, 0, 120,feature_type="histogram")

Para consultar una imagen, se utiliza la función query_image. Esta función recibe una base de datos (dataframe), la medida de similitud y el tipo de feature a utilizar. Internamente, calcula la similitud entre la imagen de consulta y el resto de las imágenes en la base de datos. Finalmente ordena el dataset según menor similitud.

In [ ]:
df_query_hand = query_image(df, IMAGE_EXAMPLE, measure=MEASURE, feature_type = "histogram")
#df_query_hand.head()

Una vez ordenado el dataset según similitud, se pueden graficar las imágenes de mayor similitud:

In [ ]:
plot_10(df_query_hand,PATH_TO_IMAGES)

#### aqui analisis del metodo

### Método por CNN

Las redes neuronales convolucionales corresponden a un tipo de aprendizaje profundo. Son consideradas altamente efectivas en el procesamiento de imágenes y videos. Este tipo de redes llevan a cabo la generalización y clasificacion gracias a la extracción de caracterísicas a través de convoluciones matemáticas. 

En general, construir una red CNN efectiva no es una tarea fácil. Es por esto que se recurrió a la internet para buscar algun modelo pre-entrenado que pudiera ser utilizado en la tarea de busqueda de contenido por imágenes propuesta en el enunciado. 

Tomando esto en consideración, para esta sección del proyecto se trabaja con la red entrenada disponible en [este repositorio](https://github.com/USCDataScience/Image-Similarity-Deep-Ranking) y el paper asociado se puede encontrar [aquí](https://github.com/USCDataScience/Image-Similarity-Deep-Ranking/blob/master/deep_ranking.pdf). *Image-Similarity-Deep-Ranking* es una red entrenada para encontrar similitudes entre imágenes a través de Deep Learning. Como CBIR es una problemática asociada a encontrar imágenes similares, y no el clasificar imágenes, se puede utilizar esta arquitectura disponible para generar histogramas. El input de la red son las imágenes de INRIA Holiday y GPR1200, la salida corresponde a un vector característico de largo 4096.


In [ ]:
print(type(df['features_CNN'][400]))

In [ ]:
plot_histogram(df,0,n_bins=4096, feature_type='CNN')

In [ ]:
from tqdm import notebook
for index,row in notebook.tqdm(df.iterrows()):
    df.at[index, 'features_CNN'] = get_hist_from_str(df.at[index, 'features_CNN'], separator=',')

In [ ]:
df_query_CNN = query_image(df,IMAGE_EXAMPLE, measure=MEASURE, feature_type = "CNN")


In [ ]:
plot_10(df_query_CNN,PATH_TO_IMAGES)

Al igual que para el método handcrafted, se utiliza la funcion query_image para consultar por una imagen a elección.

### Metodo fusion

In [ ]:
df_fusion_query = query_image(df, IMAGE_EXAMPLE, measure=MEASURE, feature_type = "fusion")

In [ ]:
plot_10(df_fusion_query,PATH_TO_IMAGES)

## Comparación de los metodos y resultados


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
path_to_images = '/root/labint/LabInt/LabInt/data/GPR1200/images'
 
# Create a figure with three rows and ten columns
fig, axes = plt.subplots(nrows=3, ncols=10, figsize=(15, 4))

# Iterate over each row and column to plot the images
for i in range(3):
    for j in range(10):
        if i == 0:
            axes[i, j].imshow(Image.open(f'{path_to_images}/{df_query_hand["image_name"][j]}'))
        elif i == 1:
            axes[i, j].imshow(Image.open(f'{path_to_images}/{df_query_CNN["image_name"][j]}'))
        else:
            axes[i, j].imshow(Image.open(f'{path_to_images}/{df_fusion_query["image_name"][j]}'))
        axes[i, j].axis('off')

# Set the title for each row
axes[0, 0].set_title('Handcrafted Features')
axes[1, 0].set_title('CNN Features')
axes[2, 0].set_title('Fused Features')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

## Conclusiones